## Accesibilidad a puntos de oferta agroecologica

In [ ]:
import geopandas as gpd
import urbanpy as up
from tqdm import tqdm
import pandas as pd

In [ ]:
tqdm.pandas()  # Activate progress bar for pandas apply

In [ ]:
biopoints = gpd.read_file("outputs/data/bio_points_w_wong.geojson")

In [ ]:
biopoints.shape

In [ ]:
lima_hexs = gpd.read_file("inputs/lima_hexs9_complete.geojson")

In [ ]:
biopoints.head()

In [ ]:
def accessibility_analysis(hexagons, pois):
    # Calculate the Nearest Facility for each Hexagon
    hexagons["lon"] = hexagons.geometry.centroid.x
    hexagons["lat"] = hexagons.geometry.centroid.y
    pois["lon"] = pois.geometry.x
    pois["lat"] = pois.geometry.y

    dists, ixs = up.utils.nn_search(
        tree_features=pois[["lat", "lon"]].values,
        query_features=hexagons[["lat", "lon"]].values,
        metric="haversine",
    )

    hexagons["nearest_poi_ix"] = ixs

    # Calculate travel times and distances
    distance_duration = hexagons.progress_apply(
        lambda row: up.routing.osrm_route(
            origin=row.geometry.centroid,
            destination=pois.iloc[row["nearest_poi_ix"]]["geometry"],
        ),
        result_type="expand",
        axis=1,
    )

    # Add columns to dataframe
    hexagons["distance_to_nearest_poi"] = distance_duration[0] / 1000  # meters to km
    hexagons["duration_to_nearest_poi"] = (
        distance_duration[1] / 60
    )  # seconds to minutes

    custom_bins, custom_labels = up.utils.create_duration_labels(
        hexagons["duration_to_nearest_poi"]
    )
    hexagons["duration_to_nearest_poi_label"] = pd.cut(
        hexagons["duration_to_nearest_poi"], bins=custom_bins, labels=custom_labels
    )

    return hexagons

In [ ]:
up.routing.start_osrm_server("peru", "south-america", "foot")

In [ ]:
hexs_city_pop_access = accessibility_analysis(hexagons=lima_hexs, pois=biopoints)

In [ ]:
up.routing.stop_osrm_server("peru", "south-america", "foot")

In [ ]:
hexs_city_pop_access["duration_to_nearest_poi_label"] = hexs_city_pop_access[
    "duration_to_nearest_poi_label"
].astype("str")

In [ ]:
hexs_city_pop_access.to_file("outputs/lima_hexs9_distance_to_bio.geojson")